# Azure AI Search с интеграция на NVIDIA NIM и LlamaIndex

В този notebook ще демонстрираме как да използваме AI моделите на NVIDIA и LlamaIndex, за да създадем мощен Retrieval-Augmented Generation (RAG) pipeline. Ще използваме LLMs и embeddings на NVIDIA, ще ги интегрираме с Azure AI Search като векторен хранилище и ще изпълним RAG за подобряване на качеството и ефективността на търсенето.

## Ползи
- **Мащабируемост**: Използвайте големите езикови модели на NVIDIA и Azure AI Search за мащабируемо и ефективно извличане.
- **Икономичност**: Оптимизирайте търсенето и извличането с ефективно векторно хранилище и хибридни техники за търсене.
- **Висока производителност**: Комбинирайте мощни LLMs с векторизирано търсене за по-бързи и точни отговори.
- **Качество**: Поддържайте високо качество на търсенето, като обосновавате отговорите на LLM с подходящо извлечени документи.

## Предпоставки
- 🐍 Python 3.9 или по-нова версия
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API ключ за достъп до LLMs и Embeddings на NVIDIA чрез микросървисите NVIDIA NIM

## Покрити функции
- ✅ Интеграция на NVIDIA LLM (ще използваме [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ NVIDIA Embeddings (ще използваме [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Разширени режими за извличане в Azure AI Search
- ✅ Индексиране на документи с LlamaIndex
- ✅ RAG с Azure AI Search и LlamaIndex, използвайки LLMs на NVIDIA

Да започваме!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Инсталация и изисквания
Създайте Python среда, използвайки версия на Python >3.10.

## Да започваме!


За да започнете, ви е необходим `NVIDIA_API_KEY`, за да използвате моделите на NVIDIA AI Foundation:
1) Създайте безплатен акаунт с [NVIDIA](https://build.nvidia.com/explore/discover).
2) Кликнете върху избрания от вас модел.
3) Под Input, изберете раздела Python и кликнете **Get API Key**, след това кликнете **Generate Key**.
4) Копирайте и запазете генерирания ключ като NVIDIA_API_KEY. Оттам ще имате достъп до крайните точки.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## Пример за RAG с LLM и Вграждане
### 1) Инициализиране на LLM
`llama-index-llms-nvidia`, известен още като LLM конектор на NVIDIA, ви позволява да се свързвате и генерирате от съвместими модели, налични в каталога на NVIDIA API. Вижте тук списък с модели за завършване на чат: https://build.nvidia.com/search?term=Text-to-Text

Тук ще използваме **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Инициализиране на Вграждането
`llama-index-embeddings-nvidia`, известен още като конектор за вграждания на NVIDIA, ви позволява да се свързвате и генерирате от съвместими модели, налични в каталога на NVIDIA API. Ние избрахме `nvidia/nv-embedqa-e5-v5` като модел за вграждане. Вижте тук списък с модели за текстово вграждане: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) Създайте Azure AI Search Vector Store


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Създаване на механизъм за заявки за задаване на въпроси върху вашите данни

Ето пример за заявка, използваща чисто търсене по вектори в Azure AI Search и обвързваща отговора с нашия LLM (Phi-3.5-MOE)


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Ето заявка, използваща хибридно търсене в Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Анализ на търсене във вектори
Отговорът на LLM точно обхваща ключовите икономически последствия, споменати в изходния текст относно руския фондов пазар. По-конкретно, се посочва, че руският фондов пазар е претърпял значителен спад, губейки 40% от стойността си, и че търговията е била спряна поради текущата ситуация. Този отговор добре съответства на предоставената информация в източника, което показва, че LLM правилно е идентифицирал и обобщил съответните детайли относно въздействието върху фондовия пазар в резултат на действията на Русия и наложените санкции.

#### Коментар върху изходните възли
Изходните възли предоставят подробен отчет за икономическите последствия, които Русия е понесла вследствие на международните санкции. Текстът подчертава, че руският фондов пазар е загубил 40% от стойността си и че търговията е била спряна. Освен това се споменават други икономически последици, като обезценяването на рублата и по-широката изолация на руската икономика. Отговорът на LLM ефективно извлича основните точки от тези възли, съсредоточавайки се върху въздействието върху фондовия пазар, както е поискано в запитването.


Сега, нека разгледаме запитване, при което Хибридното търсене не предоставя добре обоснован отговор:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Хибридно търсене: Анализ на отговорите от LLM
Отговорът на LLM в примера за хибридно търсене показва, че предоставеният контекст не уточнява точната дата на инвазията на Русия в Украйна. Този отговор предполага, че LLM използва наличната информация от изходните документи, но признава липсата на конкретни детайли в текста.

Отговорът е точен в идентифицирането, че контекстът споменава събития, свързани с агресията на Русия, но не посочва конкретната дата на инвазията. Това демонстрира способността на LLM да разбира предоставената информация, като същевременно разпознава пропуските в съдържанието. LLM ефективно насочва потребителя да потърси външни източници или исторически записи за точната дата, показвайки предпазливост, когато информацията е непълна.

### Анализ на изходните възли
Изходните възли в примера за хибридно търсене съдържат откъси от реч, обсъждаща реакцията на САЩ спрямо действията на Русия в Украйна. Тези възли подчертават по-широкото геополитическо въздействие и предприетите стъпки от САЩ и техните съюзници в отговор на инвазията, но не споменават конкретната дата на инвазията. Това съответства на отговора на LLM, който правилно идентифицира, че контекстът липсва точна информация за датата.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Хибриден модел с повторно класиране: Анализ на отговорите на LLM
В примера с хибридния модел с повторно класиране, отговорът на LLM предоставя допълнителен контекст, като отбелязва, че събитието се е случило шест дни преди речта. Това показва, че LLM е способен да направи извод за датата на инвазията въз основа на времето на речта, въпреки че все още е необходимо да знае точната дата на речта за прецизност.

Този отговор демонстрира подобрена способност за използване на контекстуални улики, за да предостави по-информативен отговор. Той подчертава предимството на повторното класиране, при което LLM може да получи достъп до по-релевантна информация и да я приоритизира, за да даде по-точно приближение на търсения детайл (т.е. датата на инвазията).

### Анализ на изходните възли
Изходните възли в този пример включват препратки към времето на инвазията на Русия, като конкретно се споменава, че тя се е случила шест дни преди речта. Въпреки че точната дата все още не е изрично посочена, възлите предоставят времеви контекст, който позволява на LLM да даде по-нюансиран отговор. Включването на този детайл демонстрира как повторното класиране може да подобри способността на LLM да извлича и прави изводи от предоставения контекст, което води до по-точен и информативен отговор.


**Забележка:**
В този тефтер използвахме микроуслуги NVIDIA NIM от каталога на NVIDIA API.  
Гореспоменатите API, `NVIDIA (llms)`, `NVIDIAEmbedding` и [Azure AI Search Semantic Hybrid Retrieval (вградено повторно класиране)](https://learn.microsoft.com/azure/search/semantic-search-overview).  
Имайте предвид, че гореспоменатите API могат също да поддържат самостоятелно хоствани микроуслуги.

**Пример:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматичните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия изходен език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален превод от човек. Ние не носим отговорност за каквито и да е недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
